In [1]:
!pip3 install wget
!pip3 install folium==0.7.0

In [2]:
import folium
import wget
import pandas as pd

In [3]:
from folium.plugins import MarkerCluster
from folium.plugins import MousePosition
from folium.features import DivIcon

In [6]:
spacex_csv_file = wget.download('https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/spacex_launch_geo.csv')
df=pd.read_csv(spacex_csv_file)

In [38]:
df.head()

,Launch Site,Lat,Long,class
0,CCAFS LC-40,28.562302,-80.577356,0
1,CCAFS LC-40,28.562302,-80.577356,0
2,CCAFS LC-40,28.562302,-80.577356,0
3,CCAFS LC-40,28.562302,-80.577356,0
4,CCAFS LC-40,28.562302,-80.577356,0


In [17]:
df = df[['Launch Site','Lat','Long','class']]
ls_df = df.groupby(['Launch Site'], as_index=False).first()

,Launch Site,Lat,Long,class
0,CCAFS LC-40,28.562302,-80.577356,0
1,CCAFS SLC-40,28.563197,-80.576820,1
2,KSC LC-39A,28.573255,-80.646895,1
3,VAFB SLC-4E,34.632834,-120.610745,0


In [19]:
ls_df.drop(columns=['class'],inplace=True)

In [20]:
nasa_coordinate = [29.559684888503615, -95.0830971930759]
site_map = folium.Map(location=nasa_coordinate, zoom_start=10)

In [23]:
circle = folium.Circle(nasa_coordinate, radius=1000, color='#d35400', fill=True).add_child(folium.Popup('NASA Johnson Space Center'))

marker = folium.map.Marker(nasa_coordinate,
                           icon = DivIcon(
                               icon_size=(20,20),
                               icon_anchor=(0,0),
                               html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % 'NASA JSC',
                           )
                          )

site_map.add_child(circle)
site_map.add_child(marker)

In [31]:
# instance = folium.map.FeatureGroup()
tmap = folium.Map(location=nasa_coordinate, zoom_start=4)

for label,lat,lon in zip(ls_df['Launch Site'],ls_df.Lat,ls_df.Long):
    folium.CircleMarker(
        [lat,lon],
        radius=5,
        color='#d35400',
        fill=True
    ).add_child(folium.Popup(label)).add_to(tmap)
    
    folium.map.Marker(
    [lat,lon],icon=DivIcon(
        icon_size=(10,10),icon_anchor=(0,0),html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % label
    )).add_to(tmap)
    
tmap

In [32]:
df.tail()

,Launch Site,Lat,Long,class
51,CCAFS SLC-40,28.563197,-80.57682,0
52,CCAFS SLC-40,28.563197,-80.57682,0
53,CCAFS SLC-40,28.563197,-80.57682,0
54,CCAFS SLC-40,28.563197,-80.57682,1
55,CCAFS SLC-40,28.563197,-80.57682,0


In [33]:
mcluster = MarkerCluster()

In [36]:
def mcolor(val):
    if val == 1:
        return 'green'
    else:
        return 'red'

In [41]:
df['marker_color'] = df['class'].apply(mcolor)
df.tail(10)

,Launch Site,Lat,Long,class,marker_color
46,KSC LC-39A,28.573255,-80.646895,1,green
47,KSC LC-39A,28.573255,-80.646895,1,green
48,KSC LC-39A,28.573255,-80.646895,1,green
49,CCAFS SLC-40,28.563197,-80.576820,1,green
50,CCAFS SLC-40,28.563197,-80.576820,1,green
51,CCAFS SLC-40,28.563197,-80.576820,0,red
52,CCAFS SLC-40,28.563197,-80.576820,0,red
53,CCAFS SLC-40,28.563197,-80.576820,0,red
54,CCAFS SLC-40,28.563197,-80.576820,1,green
55,CCAFS SLC-40,28.563197,-80.576820,0,red


In [42]:
sitemap = folium.Map(location=nasa_coordinate,zoom_start=4)


In [43]:
sitemap.add_child(mcluster)

In [48]:
for index,record in df.iterrows():
    # print(index,record.Long)
    marker = folium.Marker([record.Lat,record.Long],
                           icon=folium.Icon(
                                color='white', icon_color=record.marker_color)
                          )
    mcluster.add_child(marker)

In [49]:
sitemap

In [51]:
# Add Mouse Position to get the coordinate (Lat, Long) for a mouse over on the map
formatter = "function(num) {return L.Util.formatNum(num, 5);};"
mouse_position = MousePosition(
    position='topright',
    separator=' Long: ',
    empty_string='NaN',
    lng_first=False,
    num_digits=20,
    prefix='Lat:',
    lat_formatter=formatter,
    lng_formatter=formatter,
)

sitemap.add_child(mouse_position)
sitemap

In [52]:
from math import sin, cos, sqrt, atan2, radians

def calculate_distance(lat1, lon1, lat2, lon2):
    # approximate radius of earth in km
    R = 6373.0

    lat1 = radians(lat1)
    lon1 = radians(lon1)
    lat2 = radians(lat2)
    lon2 = radians(lon2)

    dlon = lon2 - lon1
    dlat = lat2 - lat1

    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))

    distance = R * c
    return distance

In [56]:
lat = 28.56367
long = -80.57163
distance_coastline = calculate_distance(28.573255, -80.646895, lat, long)
distance_coastline

7.42932194658424